In [1]:
import requests
import json
import pandas as pd
import re
import joblib
import os
import sys

from datetime import datetime
from bs4 import BeautifulSoup

sys.path.append('../')
from src.utilities.config_ import train_data_path, scrape_data_path, model_path, predicted_data_path

In [19]:
# read dfx df
dailyfx = pd.read_csv(os.path.join(predicted_data_path, "dailyfx_result_test.csv"))
dailyfx["source"] = "dailyfx"
dailyfx

,title,date,url,category,label,source
0,Crude Oil Q3 Technical Forecast: Narrowing Pri...,2024-06-29,https://www.dailyfx.com/analysis/crude-oil-q3-...,forex,neutral,dailyfx
1,Equities Q3 Fundamental Outlook: Bullish Momen...,2024-06-29,https://www.dailyfx.com/news/equities-q3-funda...,forex,neutral,dailyfx
2,British Pound Q3 Technical Forecast: GBP/USD E...,2024-06-29,https://www.dailyfx.com/analysis/british-pound...,forex,neutral,dailyfx
3,US Dollar Q3 Fundamental Outlook: US Dollar to...,2024-06-28,https://www.dailyfx.com/news/us-dollar-q3-fund...,forex,negative,dailyfx
4,Australian Dollar Q3 Technical Forecast,2024-06-28,https://www.dailyfx.com/analysis/australian-do...,forex,neutral,dailyfx
...,...,...,...,...,...,...
102,"​​​FTSE 100, DAX 40 and S&P 500 Recover from L...",2024-06-03,https://www.dailyfx.com/analysis/ftse-100-dax-...,forex,neutral,dailyfx
103,"GBP/USD Testing 1.2700 as USD Picks Up a Bid, ...",2024-06-03,https://www.dailyfx.com/news/gbp-usd-testing-1...,forex,neutral,dailyfx
104,OPEC+ Fights Declining Oil Prices with Extende...,2024-06-03,https://www.dailyfx.com/news/opec-fights-decli...,forex,neutral,dailyfx
105,"Markets Week Ahead: ECB Rate Decision, US NFPs...",2024-06-02,https://www.dailyfx.com/news/markets-week-ahea...,forex,neutral,dailyfx


In [20]:
# read econtimes df
econtimes = pd.read_csv(os.path.join(predicted_data_path, "econtimes_result_test.csv"))
econtimes["source"] = "econtimes"
econtimes

,title,date,url,category,label,source
0,Insurers worried over surrender value rule,2024-06-01,https://economictimes.indiatimes.com//industry...,banking,neutral,econtimes
1,PFRDA reviews strategies for expansion and dis...,2024-06-01,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
2,Centre to take decisive steps to strengthen ba...,2024-06-01,https://economictimes.indiatimes.com//industry...,banking,positive,econtimes
3,"India's fiscal numbers, growth to be monitored...",2024-06-01,https://economictimes.indiatimes.com//news/eco...,economy,positive,econtimes
4,"Economy expands 7.8% in Q4, lifting FY24 growt...",2024-06-01,https://economictimes.indiatimes.com//news/eco...,economy,positive,econtimes
...,...,...,...,...,...,...
631,Exporters' body urges Piyush Goyal to restore ...,2024-06-29,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
632,Government panel recommends Challa Sreenivasul...,2024-06-29,https://economictimes.indiatimes.com//industry...,banking,neutral,econtimes
633,Confident of over $800 billion exports in good...,2024-06-29,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
634,Confident of $800 billion exports in FY25: Piy...,2024-06-30,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes


In [24]:
combined = pd.concat([dailyfx, econtimes]).drop_duplicates().reset_index(drop=True)
combined

,title,date,url,category,label,source
0,Crude Oil Q3 Technical Forecast: Narrowing Pri...,2024-06-29,https://www.dailyfx.com/analysis/crude-oil-q3-...,forex,neutral,dailyfx
1,Equities Q3 Fundamental Outlook: Bullish Momen...,2024-06-29,https://www.dailyfx.com/news/equities-q3-funda...,forex,neutral,dailyfx
2,British Pound Q3 Technical Forecast: GBP/USD E...,2024-06-29,https://www.dailyfx.com/analysis/british-pound...,forex,neutral,dailyfx
3,US Dollar Q3 Fundamental Outlook: US Dollar to...,2024-06-28,https://www.dailyfx.com/news/us-dollar-q3-fund...,forex,negative,dailyfx
4,Australian Dollar Q3 Technical Forecast,2024-06-28,https://www.dailyfx.com/analysis/australian-do...,forex,neutral,dailyfx
...,...,...,...,...,...,...
738,Exporters' body urges Piyush Goyal to restore ...,2024-06-29,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
739,Government panel recommends Challa Sreenivasul...,2024-06-29,https://economictimes.indiatimes.com//industry...,banking,neutral,econtimes
740,Confident of over $800 billion exports in good...,2024-06-29,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
741,Confident of $800 billion exports in FY25: Piy...,2024-06-30,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes


In [32]:
pd.concat([combined, dailyfx]).drop_duplicates().reset_index(drop=True)

,title,date,url,category,label,source
0,Crude Oil Q3 Technical Forecast: Narrowing Pri...,2024-06-29,https://www.dailyfx.com/analysis/crude-oil-q3-...,forex,neutral,dailyfx
1,Equities Q3 Fundamental Outlook: Bullish Momen...,2024-06-29,https://www.dailyfx.com/news/equities-q3-funda...,forex,neutral,dailyfx
2,British Pound Q3 Technical Forecast: GBP/USD E...,2024-06-29,https://www.dailyfx.com/analysis/british-pound...,forex,neutral,dailyfx
3,US Dollar Q3 Fundamental Outlook: US Dollar to...,2024-06-28,https://www.dailyfx.com/news/us-dollar-q3-fund...,forex,negative,dailyfx
4,Australian Dollar Q3 Technical Forecast,2024-06-28,https://www.dailyfx.com/analysis/australian-do...,forex,neutral,dailyfx
...,...,...,...,...,...,...
738,Exporters' body urges Piyush Goyal to restore ...,2024-06-29,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
739,Government panel recommends Challa Sreenivasul...,2024-06-29,https://economictimes.indiatimes.com//industry...,banking,neutral,econtimes
740,Confident of over $800 billion exports in good...,2024-06-29,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
741,Confident of $800 billion exports in FY25: Piy...,2024-06-30,https://economictimes.indiatimes.com//news/eco...,economy,neutral,econtimes
